In [ ]:
!pip install pyyaml==5.1

import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html
# If there is not yet a detectron2 release that matches the given torch + CUDA version, you need to install a different pytorch.

# exit(0)  # After installation, you may need to "restart runtime" in Colab. This line can also restart runtime

     |████████████████████████████████| 274 kB 10.9 MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44092 sha256=97628b498f0de2fa08ad4562282a71e49c8c84cb7cf60b3a465cf7b4652129cd
  Stored in directory: /root/.cache/pip/wheels/77/f5/10/d00a2bd30928b972790053b5de0c703ca87324f3fead0f2fd9
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
torch:  1.10 ; cuda:  cu111
Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.10/index.html
     |████████████████████████████████| 7.0 MB 718 kB/s 
     |████████████████████████████████| 130 kB 12.7 MB/s 
     |████████████████████████████████| 145 kB 37.5 MB/s 
     |████████████████████████████████| 49 kB 5.5 MB/s 
     |████████████████████████████████| 74 kB 3.4 MB/s 
     |████████████████████████████████| 749 kB 42.2 MB/s 
     |██████████████████████████████

In [ ]:
import os
from google.colab import drive
os.chdir("/content")

drive._mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.chdir('/content')
from google_drive_downloader import GoogleDriveDownloader as gdd
gdd.download_file_from_google_drive(file_id='1nEJ7NTtHcCHNQqUXaoPk55VH3Uwh4QGG',
                                    dest_path='./data.zip',
                                    unzip=True)

Unzipping...Done.


In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
data_path = os.listdir('/content/dataset/train')

train_data = data_path[:20]
val_data = data_path[20:]
image_ids = train_data
VAL_IMAGE_IDS = val_data
os.makedirs('/content/dataset/coco')
os.makedirs('/content/dataset/coco/annotation')
os.makedirs('/content/dataset/coco/train')
os.makedirs('/content/dataset/coco/val')
import shutil
shutil.move('/content/dataset/test','/content/dataset/coco')

for i in train_data:
  shutil.move('/content/dataset/train/' + f'{i}' + '/images/' + f'{i}' + '.png','/content/dataset/coco/train')
for i in val_data:
  shutil.move('/content/dataset/train/' + f'{i}' + '/images/' + f'{i}' + '.png','/content/dataset/coco/val')

shutil.move('/content/dataset/test_img_ids.json','/content/dataset/coco/annotation')
shutil.copy('/content/drive/MyDrive/Colab/VRDL/hw3/new/train.json','/content/dataset/coco/annotation')
shutil.copy('/content/drive/MyDrive/Colab/VRDL/hw3/new/val.json','/content/dataset/coco/annotation')

shutil.rmtree('/content/dataset/train')

In [ ]:
import json 

with open('/content/dataset/coco/annotation/test_img_ids.json') as f:
    data = json.load(f)

In [ ]:
for i in data:
  i['file_name'] = '/content/dataset/coco/test/'+i['file_name']

for i in data:
  i['image_id'] = i['id'] - 1


In [ ]:
classes = ["nucleus"]
d = 5
DatasetCatalog.register("nucleus_test", lambda d=d: data)
MetadataCatalog.get("nucleus_test").set(thing_classes=classes)

Metadata(name='nucleus_test', thing_classes=['nucleus'])

In [ ]:
from detectron2.engine import DefaultPredictor, DefaultTrainer
class CocoTrainer(DefaultTrainer):
  
  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):

    if output_folder is None:
        os.makedirs("coco_eval", exist_ok=True)
        output_folder = "coco_eval"

    return COCOEvaluator(dataset_name, cfg, False, output_folder)

In [ ]:
cfg = get_cfg()
#cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("nucleus_train",)
cfg.DATASETS.TEST = ("nucleus_val",)
cfg.DATALOADER.NUM_WORKERS = 4
#cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025   # pick a good LR
cfg.SOLVER.MAX_ITER = 3000    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = [1000, 2000]        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 32   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.
cfg.TEST.EVAL_PERIOD = 100
cfg.MODEL.ANCHOR_GENERATOR.SIZES = [[8], [16], [32], [64], [128]]
cfg.SOLVER.WARMUP_ITERS = 1000
cfg.SOLVER.STEPS = []
cfg.SOLVER.GAMMA = 0.05
cfg.MODEL.WEIGHTS = '/content/drive/MyDrive/model_final.pth'
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.001   # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("nucleus_test", cfg, False, output_dir="./output_test/")
val_loader = build_detection_test_loader(cfg, "nucleus_test")
inference_on_dataset(predictor.model, val_loader, evaluator)

WARNING [12/14 15:08:59 d2.evaluation.coco_evaluation]: COCO Evaluator instantiated using config, this is deprecated behavior. Please pass in explicit arguments instead.
[12/14 15:08:59 d2.evaluation.coco_evaluation]: Trying to convert 'nucleus_test' to COCO format ...
[12/14 15:08:59 d2.data.datasets.coco]: Converting annotations of dataset 'nucleus_test' to COCO format ...)
[12/14 15:08:59 d2.data.datasets.coco]: Converting dataset dicts into COCO format
[12/14 15:08:59 d2.data.datasets.coco]: Conversion finished, #images: 6, #annotations: 0
[12/14 15:08:59 d2.data.datasets.coco]: Caching COCO format annotations at './output_test/nucleus_test_coco_format.json' ...
[12/14 15:08:59 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[12/14 15:08:59 d2.data.common]: Serializing 6 elements to byte tensors and concatenating them all ...
[12/14 15:08:59 d2.data.common]: Serialized

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride
/usr/local/lib/python3.7/dist-p

[12/14 15:09:03 d2.evaluation.evaluator]: Total inference time: 0:00:00.692514 (0.692514 s / iter per device, on 1 devices)
[12/14 15:09:03 d2.evaluation.evaluator]: Total inference pure compute time: 0:00:00 (0.346889 s / iter per device, on 1 devices)
[12/14 15:09:03 d2.evaluation.coco_evaluation]: Preparing results for COCO format ...
[12/14 15:09:03 d2.evaluation.coco_evaluation]: Saving results to ./output_test/coco_instances_results.json
[12/14 15:09:03 d2.evaluation.coco_evaluation]: Annotations are not available for evaluation.


OrderedDict()

In [ ]:
with open('/content/output_test/coco_instances_results.json') as f:
    data = json.load(f)

for i in data:
  i['category_id'] = i['category_id'] + 1,
  i['category_id'] = i['category_id'][0]
  i['image_id'] = i['image_id'] + 1

In [ ]:
data[0]

{'bbox': [443.222412109375,
  56.41257858276367,
  23.469635009765625,
  18.054386138916016],
 'category_id': 1,
 'image_id': 1,
 'score': 0.9736591577529907,
 'segmentation': {'counts': 'ge`=6Po05M1N2N100O100000000O10000000000000O100O1O2N2M_^X`0',
  'size': [1000, 1000]}}

In [ ]:
with open("answer.json", "w") as f:
    json.dump(data, f)